## Simple Network Example

This notebook demonstrates the basic functionality of the repository: The demand of a load has to be met over time, given imperfect predictions of demand and renewable generation.

In [ ]:
import numpy as np
import pandas as pd
import pypsa
import matplotlib.pyplot as plt
plt.style.use('bmh')
import os

os.chdir(os.path.join(os.getcwd(), '..', 'src'))
from mpc import Controller

First, create a network with a load, a dispatchable but expensive and a non-dispatchable but cheap generation unit.

In [ ]:
def make_small_network():
    '''
    creates 3 bus network:
    house (predicted demand)
    wind farm (predicted supply)
    plant (fixed high price)
    '''

    network = pypsa.Network()

    network.add('Bus', 'bus0')
    network.add('Bus', 'bus1')
    network.add('Bus', 'bus2')

    # network.add('Load', 'house', bus='bus0', p_nom=1, p_set=pd.Series(0.5*np.ones(len(snapshots))))
    
    network.add('Load', 'house', bus='bus0', p_set=0.5)
    network.add('Generator', 'pv', bus='bus1', p_nom=1.,
                        ramp_limit_up=0.2, ramp_limit_down=0.2, marginal_cost=0.)
    network.add('Generator', 'plant', bus='bus2', p_nom=1., marginal_cost=1.,
                        ramp_limit_up=0.2, ramp_limit_down=0.2)

    network.add('Link', 'pv link', bus0='bus1', bus1='bus0',
                efficiency=1., p_nom=1)
    network.add('Link', 'plant link', bus0='bus2', bus1='bus0',
                efficiency=1., p_nom=1)

    return network


Define the parameters of the optimization

In [ ]:
t_steps = 48
total_snapshots = pd.date_range('2020-01-01', '2020-02-01', freq='30min')[:t_steps]
horizon = 13

Setup the config of the predictions:

For each prediction a Prophet-object will be instantiated. In this example, they are not real machine learning models but read off data and add artificial noise.

1. For the pv unit we predict __p_max_pu__. The respective data is in data/dummy/supply.csv
2. The load is predicted as the __p_set__.
3. The marginal price of generation is set as fixed

In [ ]:
data_path = os.path.join(os.getcwd(), '..', 'data', 'dummy')

prophets_config = {
        'pv': [
                {
                'kind': 'p_max_pu', 
                'mode': 'read', 
                'noise_scale': 0.02,
                'data': os.path.join(data_path, 'supply.csv')
                },
                ],
        'house': [
                {
                'kind': 'p_set', 
                'mode': 'read',
                'noise_scale': 0.005, 
                'data': os.path.join(data_path, 'demand.csv')
                },
                ],
        'plant': [
                {
                'kind': 'marginal_cost',
                'mode': 'fix',
                'value': 1. 
                }
                ]
        }


In [ ]:
init_values = {'pv': 0., 'plant': 0.5, 'house': 0.5}

Define controller object

In [ ]:
mpc = Controller(
        make_small_network(),
        total_snapshots,
        prophets_config,
        horizon,
        init_values=init_values,
        solver_name='gurobi' # or any solver installed on your machine
        )

Start control loop - Passing an axis object into the procedure creates plots of the result.

In [ ]:
_, ax= plt.subplots(1, 1, figsize=(16, 4))

# plotting ground truth, from which the prophets are reading
for _, prophet in mpc.prophets.items():
    prophet.data.plot(ax=ax)
    
for time in range(t_steps - horizon - 1):
    
    print(f'Conducting optimization step at {time}.')

    # Define the snapshots of the current iteration
    snapshots = total_snapshots[time:time+horizon+1]

    # Note that at the moment, we pass a network generating function
    # instead of the network itself
    mpc.mpc_step(make_small_network,
                snapshots, 
                ax=ax)

# plot the chosen controls
mpc.controls_t[init_values].plot(ax=ax, linestyle=':')

ax.set_ylabel('Power Flow')
ax.set_xlabel('Time')

plt.show()

The thick lines represent the ground truth for demand and renewable generation, while the thin lines show the time series returned by the prophets and used in the individual optimization steps.

The thick dotted lines are the controls chosen at each timestep. Note that the features at the start of the optimization are the result of initial conditions paired with limit on the production ramps. 